**Goal:** represent the tracks downloaded from [Wikiloc](http://wikiloc.com/) or transformed into .gpx format from Garmin .fit files, using for example the [gpsbabel](http://www.gpsbabel.org/) tool.

In [1]:
import re
import os
import folium
import glob
import logging
import numpy as np
from geopy.geocoders import Nominatim
from folium import plugins

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# User inputs

Define the directory where the files are stored and where the html will be written.

In [14]:
region = 'SOCIBhiking'
gpxdir = '/home/ctroupin/DataOceano/GPX/' + region 
outputdir = '/home/ctroupin/Documents/Trails/LeafletMaps'
mapname = region + '.html'
gpxbasename = '*.gpx'
gpxfilelist = sorted(glob.glob(os.path.join(gpxdir, gpxbasename)))

Initialise the map

In [9]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='./log/GPXheadmap.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

geolocator = Nominatim()
location = geolocator.geocode("Lloret de Vistalegre, Mallorca, Spain")
#location = geolocator.geocode("Verviers, Belgique")
#location = geolocator.geocode("Gran Canaria, Spain")
logger.info("Working on location: %s" %(location[0]))

Function to read GPX

In [10]:
def read_gpx(filename):
    coords = []
    with open(filename) as f:
        for lines in f:
            match = re.search('<trkpt lat="([-0-9\.]+)" lon="([-0-9\.]+)">', lines)
            if match:
                coords.append((float(match.group(1)), float(match.group(2))))
    return coords

In [11]:
def get_trackname(filename):
    with open(filename) as f:
        match = None
        while match == None:
            line = f.readline()
            match = re.search('<name>(.+)</name>', line)
            if match:
                trackname = match.group(1)
    return trackname

Loop on the files

In [12]:
logger.info('Creating a new map')
map_run = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=10)
iconurl = 'http://socib.es/users/ctroupin/LeafletMaps/glider.png'

coordsall = []
logger.info('Loop on the GPX files')
for gpxfiles in gpxfilelist:
    
    trackname = get_trackname(gpxfiles)
    logger.debug(trackname)
    coords = read_gpx(gpxfiles)
    coordsall.append(coords)
    folium.PolyLine(coords, color="black", weight=3)
    icon = folium.features.CustomIcon(iconurl, icon_size=(40, 40))
    folium.Marker(location=coords[0], popup=trackname, icon=icon).add_to(map_run)
    
coords2plot = [item for coords in coordsall for item in coords] 
folium.plugins.HeatMap(coords2plot, radius=5, blur=5).add_to(map_run)
map_run

In [15]:
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    logging.info('Creating directory %s' %(outputdir))
map_run.save(os.path.join(outputdir, mapname))